In [20]:
import xml.etree.ElementTree as xml
from glob import glob
import os
from tqdm import tqdm

In [4]:
raw_data_path = r"../data/raw/*"
processed_data_path = '../data/processed'

In [22]:
if not os.path.isdir(processed_data_path):
    os.mkdir(processed_data_path)

In [10]:
path = glob(raw_data_path)
# path[0]
# print(path[0].split('/')[-1][:-4])

subject7072


## Run the script to generate `.txt` files from `.xml` documents

### TODO:

- replace URLs with an appropiate `<TAG>` 
- there are some portions of text like this N #176;1, it needs to be changed

In [23]:
for p in tqdm(path):
    subject = p.split('/')[-1][:-4] + ".txt"
    # print(subject)
    subject_path = os.path.join(processed_data_path, subject)
    if os.path.isfile(subject_path):
        continue
    root = xml.parse(p).getroot()
    for text in root.findall("WRITING/TEXT"):
        if text.text is not None:
            with open(subject_path, 'a') as f:
                f.write(text.text)
                f.write("\n")

100%|██████████| 2348/2348 [00:28<00:00, 81.69it/s] 
